In [1]:
from pathlib import Path
from string import punctuation
import warnings

import numpy as np
import psycopg2 as pg

import database
import extract_features

In [2]:
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [3]:
from importlib import reload
reload(database)
reload(extract_features);

In [4]:
cwd = Path.cwd()
seg_npz = cwd / 'data' / 'seg_npz'
all_segments = sorted(seg for seg in seg_npz.iterdir() if seg.suffix == '.npz')

In [5]:
len(all_segments)

283

In [ ]:
from datetime import datetime
punct_table = str.maketrans({char: None for char in punctuation + ' '})

for i, segments_npz in enumerate(all_segments[37:], start=37):  # change limits! also print times at intervals
    segments, name = np.load(segments_npz).values()
    name = np.asscalar(name).translate(punct_table)

    print(i, datetime.now().strftime("%A, %d. %B %Y %I:%M%p"), '\n')
    for seg_id, seg_audio in enumerate(segments):
        print(name, seg_id)
        feature_dct = extract_features.extract_all(seg_audio)
        database.insert_features(name, seg_id, feature_dct)
    print('='*20)

37 Sunday, 10. February 2019 09:31PM 

RememberMe 0
RememberMe 1
RememberMe 2
RememberMe 3
RememberMe 4
RememberMe 5
RememberMe 6
38 Sunday, 10. February 2019 09:37PM 

IMBack 0
IMBack 1
IMBack 2
IMBack 3
IMBack 4
IMBack 5
IMBack 6
IMBack 7
IMBack 8
IMBack 9
39 Sunday, 10. February 2019 09:45PM 

MarshallMathers 0
MarshallMathers 1
MarshallMathers 2
MarshallMathers 3
MarshallMathers 4
MarshallMathers 5
MarshallMathers 6
MarshallMathers 7
MarshallMathers 8
MarshallMathers 9
40 Sunday, 10. February 2019 09:53PM 

KenKaniff 0
KenKaniff 1
41 Sunday, 10. February 2019 09:55PM 

DrugBallad 0
DrugBallad 1
DrugBallad 2
DrugBallad 3
DrugBallad 4
DrugBallad 5
DrugBallad 6
DrugBallad 7
DrugBallad 8
DrugBallad 9
42 Sunday, 10. February 2019 10:03PM 

Amityville 0
Amityville 1
Amityville 2
Amityville 3
Amityville 4
Amityville 5
Amityville 6
Amityville 7
43 Sunday, 10. February 2019 10:10PM 

BitchPleaseIi 0
BitchPleaseIi 1
BitchPleaseIi 2
BitchPleaseIi 3
BitchPleaseIi 4
BitchPleaseIi 5
BitchPleaseI

RickyTickyToc 3
RickyTickyToc 4
87 Monday, 11. February 2019 02:54AM 

DrWest 0
DrWest 1
88 Monday, 11. February 2019 02:55AM 

3AM 0
3AM 1
3AM 2
3AM 3
3AM 4
3AM 5
3AM 6
3AM 7
3AM 8
3AM 9
89 Monday, 11. February 2019 03:04AM 

MyMom 0
MyMom 1
MyMom 2
MyMom 3
MyMom 4
MyMom 5
MyMom 6
MyMom 7
MyMom 8
MyMom 9
90 Monday, 11. February 2019 03:12AM 

Insane 0
Insane 1
Insane 2
Insane 3
Insane 4
Insane 5
91 Monday, 11. February 2019 03:17AM 

BagpipesFromBaghdad 0
BagpipesFromBaghdad 1
BagpipesFromBaghdad 2
BagpipesFromBaghdad 3
BagpipesFromBaghdad 4
BagpipesFromBaghdad 5
BagpipesFromBaghdad 6
BagpipesFromBaghdad 7
BagpipesFromBaghdad 8
92 Monday, 11. February 2019 03:24AM 

Hello 0
Hello 1
Hello 2
Hello 3
Hello 4
Hello 5
Hello 6
Hello 7
93 Monday, 11. February 2019 03:31AM 

Tonya 0
94 Monday, 11. February 2019 03:32AM 

SameSongAndDance 0
SameSongAndDance 1
SameSongAndDance 2
SameSongAndDance 3
SameSongAndDance 4
SameSongAndDance 5
SameSongAndDance 6
SameSongAndDance 7
95 Monday, 11. Februar

137 Monday, 11. February 2019 08:13AM 

WhatItIsAkaNinaSSongDem 0
WhatItIsAkaNinaSSongDem 1
WhatItIsAkaNinaSSongDem 2
WhatItIsAkaNinaSSongDem 3
WhatItIsAkaNinaSSongDem 4
WhatItIsAkaNinaSSongDem 5
WhatItIsAkaNinaSSongDem 6
138 Monday, 11. February 2019 08:19AM 

JungleMan 0
JungleMan 1
JungleMan 2
JungleMan 3
JungleMan 4
JungleMan 5
JungleMan 6
JungleMan 7
139 Monday, 11. February 2019 08:25AM 

HollywoodAfrica 0
HollywoodAfrica 1
HollywoodAfrica 2
HollywoodAfrica 3
HollywoodAfrica 4
HollywoodAfrica 5
HollywoodAfrica 6
HollywoodAfrica 7
HollywoodAfrica 8
HollywoodAfrica 9
140 Monday, 11. February 2019 08:34AM 

AmericanGhostDance 0
AmericanGhostDance 1
AmericanGhostDance 2
AmericanGhostDance 3
AmericanGhostDance 4
AmericanGhostDance 5
AmericanGhostDance 6
141 Monday, 11. February 2019 08:40AM 

IfYouWantMeToStay 0
IfYouWantMeToStay 1
IfYouWantMeToStay 2
IfYouWantMeToStay 3
IfYouWantMeToStay 4
IfYouWantMeToStay 5


In [24]:
keys

['rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_flatness',
 'spectral_rolloff',
 'zero_crossing_rate',
 'x_perf5',
 'y_perf5',
 'x_min3',
 'y_min3',
 'x_maj3',
 'y_maj3',
 'bound_width',
 'pitch_0',
 'pitch_1',
 'pitch_2',
 'pitch_3',
 'pitch_4',
 'pitch_5',
 'pitch_6',
 'pitch_7',
 'pitch_8',
 'pitch_9',
 'pitch_10',
 'pitch_11',
 'tempogram',
 'dynamic_tempo']